In [6]:
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential
from keras.layers import Dense, Dropout, Masking, Embedding
from matplotlib import pyplot as plt
from ipywidgets import interact
plt.rcParams['figure.dpi'] = 200

In [2]:
file_headers = ["/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve2"]

CSV_SFX = ".csv"
MARKERS_SFX = "_markers"
POLY_SFX = "_poly"
M10Y = "M10Y"

In [3]:
# distance goes from 22-38. 16 cm of extension at least which is pretty good.

In [4]:
poly_df_list = []
data_df_list = []

for header in file_headers:
    data_file = header + CSV_SFX
    marker_file = header + MARKERS_SFX + CSV_SFX
    poly_file = header + POLY_SFX + CSV_SFX
    
    data_df = pd.read_csv(data_file)
    marker_df = pd.read_csv(marker_file)
    poly_df = pd.read_csv(poly_file)
    
    zero_indices = list(marker_df[marker_df[M10Y] == 0].index)
    
    data_df = data_df.drop(data_df.index[zero_indices]).reset_index()
    poly_df = poly_df.drop(poly_df.index[zero_indices])
    
    data_df_list.append(data_df)
    poly_df_list.append(poly_df)

total_data_df = pd.concat(data_df_list).reset_index()
total_poly_df = pd.concat(poly_df_list).reset_index()

training_data_list = [total_data_df, total_poly_df]
all_df = pd.concat(training_data_list, axis=1)

x_labels = ["M1-PL", "M1-PR", "M2-PL", "M2-PR"]
y_labels = ["a0", "a1", "a2", "a3", "a4", "a5", "d"]

def normalize_columns(df, col_list):
    min_vals = []
    max_vals = []
    for c in col_list:
        min_val = df[c].min()
        max_val = df[c].max()
        min_vals.append(min_val)
        max_vals.append(max_val)
        df[c] = (df[c] - min_val) / (max_val - min_val)
    return min_vals, max_vals
        
x_mins, x_maxes = normalize_columns(all_df, x_labels)
y_mins, y_maxes = normalize_columns(all_df, y_labels)

In [9]:
# Prepare data for DNN
# shuffle the rows
#train_df = all_df.sample(frac=0.8, random_state=25)
#test_df = all_df.drop(train_df.index)

x_dim = len(x_labels)
y_dim = len(y_labels)

x_data = all_df[x_labels]
y_data = all_df[y_labels]

x_data_shuffled, y_data_shuffled = sklearn.utils.shuffle(x_data, y_data)

split = int(0.8 * len(x_data))

x_train = x_data_shuffled[:split]
y_train = y_data_shuffled[:split]

x_test = x_data_shuffled[split:]
y_test = y_data_shuffled[split:]

In [9]:
# prepare data for RNN
def expand_time_steps(data, time_steps):
    new_arr = np.zeros((data.shape[0] - time_steps, time_steps, data.shape[1]))
    for i in range(data.shape[0] - time_steps):
        new_arr[i, :, :] = data.iloc[i:i + time_steps]
    return new_arr

x_raw = all_df[x_labels]
y_raw = all_df[y_labels]

x_exp = expand_time_steps(x_raw, 32)
y_exp = y_raw[:-32]
print(x_exp.shape)
print(y_exp.shape)

(6087, 32, 4)
(6087, 7)


# DNN

In [10]:
model = Sequential()
model.add(Dense(64 , input_dim=x_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, input_dim=16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_dim, activation='relu'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train, epochs=300, batch_size=10)

Epoch 1/300
490/490 [==============================] - 0s 407us/step - loss: 0.0041 - accuracy: 0.8184
Epoch 2/300
490/490 [==============================] - 0s 412us/step - loss: 0.0041 - accuracy: 0.8215
Epoch 3/300
490/490 [==============================] - 0s 405us/step - loss: 0.0042 - accuracy: 0.8268
Epoch 4/300
490/490 [==============================] - 0s 409us/step - loss: 0.0040 - accuracy: 0.8108
Epoch 5/300
490/490 [==============================] - 0s 415us/step - loss: 0.0040 - accuracy: 0.8178
Epoch 6/300
490/490 [==============================] - 0s 421us/step - loss: 0.0041 - accuracy: 0.8131
Epoch 7/300
490/490 [==============================] - 0s 463us/step - loss: 0.0042 - accuracy: 0.8108
Epoch 8/300
490/490 [==============================] - 0s 455us/step - loss: 0.0043 - accuracy: 0.8165
Epoch 9/300
490/490 [==============================] - 0s 419us/step - loss: 0.0042 - accuracy: 0.8139
Epoch 10/300
490/490 [==============================] - 0s 451us/step - l

490/490 [==============================] - 0s 431us/step - loss: 0.0040 - accuracy: 0.8266
Epoch 80/300
490/490 [==============================] - 0s 408us/step - loss: 0.0040 - accuracy: 0.8190
Epoch 81/300
490/490 [==============================] - 0s 417us/step - loss: 0.0041 - accuracy: 0.8170
Epoch 82/300
490/490 [==============================] - 0s 408us/step - loss: 0.0039 - accuracy: 0.8174
Epoch 83/300
490/490 [==============================] - 0s 416us/step - loss: 0.0040 - accuracy: 0.8133
Epoch 84/300
490/490 [==============================] - 0s 415us/step - loss: 0.0040 - accuracy: 0.8151
Epoch 85/300
490/490 [==============================] - 0s 421us/step - loss: 0.0040 - accuracy: 0.8074
Epoch 86/300
490/490 [==============================] - 0s 418us/step - loss: 0.0040 - accuracy: 0.8129
Epoch 87/300
490/490 [==============================] - 0s 422us/step - loss: 0.0040 - accuracy: 0.8112
Epoch 88/300
490/490 [==============================] - 0s 420us/step - loss:

490/490 [==============================] - 0s 427us/step - loss: 0.0039 - accuracy: 0.8129
Epoch 158/300
490/490 [==============================] - 0s 425us/step - loss: 0.0040 - accuracy: 0.8231
Epoch 159/300
490/490 [==============================] - 0s 418us/step - loss: 0.0039 - accuracy: 0.8206
Epoch 160/300
490/490 [==============================] - 0s 417us/step - loss: 0.0039 - accuracy: 0.8143
Epoch 161/300
490/490 [==============================] - 0s 418us/step - loss: 0.0039 - accuracy: 0.8141
Epoch 162/300
490/490 [==============================] - 0s 416us/step - loss: 0.0038 - accuracy: 0.8155
Epoch 163/300
490/490 [==============================] - 0s 411us/step - loss: 0.0039 - accuracy: 0.8159
Epoch 164/300
490/490 [==============================] - 0s 408us/step - loss: 0.0040 - accuracy: 0.8153
Epoch 165/300
490/490 [==============================] - 0s 422us/step - loss: 0.0040 - accuracy: 0.8145
Epoch 166/300
490/490 [==============================] - 0s 413us/ste

490/490 [==============================] - 0s 402us/step - loss: 0.0040 - accuracy: 0.8168
Epoch 236/300
490/490 [==============================] - 0s 396us/step - loss: 0.0038 - accuracy: 0.8202
Epoch 237/300
490/490 [==============================] - 0s 561us/step - loss: 0.0040 - accuracy: 0.8143
Epoch 238/300
490/490 [==============================] - 0s 416us/step - loss: 0.0038 - accuracy: 0.8139
Epoch 239/300
490/490 [==============================] - 0s 399us/step - loss: 0.0040 - accuracy: 0.8231
Epoch 240/300
490/490 [==============================] - 0s 396us/step - loss: 0.0037 - accuracy: 0.8272
Epoch 241/300
490/490 [==============================] - 0s 476us/step - loss: 0.0038 - accuracy: 0.8198
Epoch 242/300
490/490 [==============================] - 0s 420us/step - loss: 0.0039 - accuracy: 0.8176
Epoch 243/300
490/490 [==============================] - 0s 396us/step - loss: 0.0039 - accuracy: 0.8163
Epoch 244/300
490/490 [==============================] - 0s 394us/ste

In [13]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss: %.8f, Accuracy: %.2f' % (loss, (accuracy*100)))

39/39 [==============================] - 0s 383us/step - loss: 0.0018 - accuracy: 0.8856
Loss: 0.00177368, Accuracy: 88.56


In [14]:
def rescale_output(y):
    ret = []
    for i in range(len(y)):
        max_val = y_maxes[i]
        min_val = y_mins[i]
        val = y[i]
        rescaled_val = (val * (max_val - min_val)) + min_val
        ret.append(rescaled_val)
    return np.array(ret)

In [15]:
num_tests = len(x_test)
def evaluate_poly(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = model.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    a_pred = np.flip(y_pred[:-1])
    d_pred = y_pred[-1]
    
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    y_select = rescale_output(y_select[0])
    a_select = np.flip(y_select[:-1])
    d_select = y_select[-1]
    
    print(d_pred, d_select)
    
    poly_pred = np.poly1d(a_pred)
    poly_select = np.poly1d(a_select)
    
    yp = np.linspace(0, d_pred)
    xp = poly_pred(yp)
    
    ys = np.linspace(0, d_select)
    xs = poly_select(ys)
    
    plt.plot(xs, ys)
    plt.plot(xp, yp)
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.show()
    

interact(evaluate_poly)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_poly(idx=(0, 1223, 1))>

In [149]:
model.save("./models/first_model")

INFO:tensorflow:Assets written to: ./models/first_model/assets


In [ ]:
# First model tried
model = Sequential()
model.add(Dense(12, input_dim=x_dim, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(y_dim, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])